In [69]:


import os
import re
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Tuple
from collections import defaultdict
from sklearn.model_selection import StratifiedKFold, train_test_split



# Load data for native Ru

In [2]:

age_gender = pd.read_excel("../Datasets/MECO/age_gender.xlsx")
age_gender



,uniform_id,Age,Sex,lang
0,du_4,19,0.0,du
1,du_5,21,0.0,du
2,du_6,27,0.0,du
3,du_7,23,1.0,du
4,du_8,27,0.0,du
...,...,...,...,...
592,tr_48,20,0.0,tr
593,tr_49,22,1.0,tr
594,tr_50,21,0.0,tr
595,tr_51,25,0.0,tr


In [3]:

demo = pd.read_excel("../Datasets/MECO/demo_alllang.xlsx")
demo



,subid,uniform_id,lang,L2_spelling_skill,L2_vocabulary_size,vocab.t2.5,L2_lexical_skill,TOWRE_word,TOWRE_nonword,motiv,IQ
0,DU_04,du_4,du,31.0,66.0,37.0,75.00,84.0,56.0,3.8,6.0
1,DU_05,du_5,du,35.0,78.0,36.0,93.75,63.0,54.0,4.0,4.0
2,DU_06,du_6,du,40.0,80.0,39.0,95.00,84.0,54.0,3.8,9.0
3,DU_07,du_7,du,33.0,50.0,36.0,66.25,65.0,45.0,3.5,9.0
4,DU_08,du_8,du,32.0,50.0,37.0,73.75,82.0,28.0,3.9,6.0
...,...,...,...,...,...,...,...,...,...,...,...
601,rum_67,rum_67,ru,40.0,69.0,38.0,93.75,88.0,52.0,4.0,11.0
602,rum_68,rum_68,ru,39.0,65.0,37.0,76.65,81.0,46.0,3.8,11.0
603,rum_69,rum_69,ru,37.0,66.0,37.0,68.75,84.0,51.0,3.6,9.0
604,rum_70,rum_70,ru,39.0,40.0,35.0,73.75,74.0,41.0,3.5,8.0


In [4]:


ru_data = pd.read_excel("../Datasets/MECO/DATA/ru_data.xlsx")
ru_data


,SubjectID,Text_ID,Fix_X,Fix_Y,Fix_Duration,Word_Number,Word,Sentence,Language
0,ru_10,1,138,352,299,77.0,portrait,"In 1825, Morse was i",ru
1,ru_10,1,436,413,338,97.0,ill,"In 1825, Morse was i",ru
2,ru_10,1,507,417,197,98.0,back,"In 1825, Morse was i",ru
3,ru_10,1,1072,411,111,105.0,message,The message had take,ru
4,ru_10,1,1259,408,229,107.0,taken,The message had take,ru
...,...,...,...,...,...,...,...,...,...
236684,rum_9,12,324,150,177,20.0,smart,Technology is rapidl,ru
236685,rum_9,12,126,123,269,1.0,Technology,Technology is rapidl,ru
236686,rum_9,12,355,138,209,20.0,smart,Technology is rapidl,ru
236687,rum_9,12,464,122,102,4.0,expanding,Technology is rapidl,ru


In [19]:


ru_data_subject_id = set(ru_data.SubjectID)


# Check ups

In [17]:


age_gender_ru = age_gender.loc[age_gender.lang == "ru"]
age_gender_ru_subject_id = set(age_gender_ru.uniform_id)
age_gender_ru



,uniform_id,Age,Sex,lang
357,rum_1,30,0.0,ru
358,rum_2,20,0.0,ru
359,rum_3,20,0.0,ru
360,rum_4,20,0.0,ru
361,rum_5,29,0.0,ru
...,...,...,...,...
459,ru_52,29,0.0,ru
460,ru_53,27,1.0,ru
461,ru_54,28,1.0,ru
462,ru_55,29,1.0,ru


In [16]:


demo_ru = demo.loc[demo.lang == "ru"]
demo_ru_subject_id = set(demo_ru.subid)
demo_ru



,subid,uniform_id,lang,L2_spelling_skill,L2_vocabulary_size,vocab.t2.5,L2_lexical_skill,TOWRE_word,TOWRE_nonword,motiv,IQ
430,ru_10,ru_10,ru,37.0,46.0,33.0,68.75,77.0,45.0,3.4,7.0
431,ru_11,ru_11,ru,40.0,67.0,36.0,97.50,90.0,55.0,3.4,9.0
432,ru_12,ru_12,ru,30.0,26.0,26.0,70.00,78.0,47.0,3.4,10.0
433,ru_13,ru_13,ru,37.0,69.0,39.0,81.25,89.0,54.0,3.6,10.0
434,ru_14,ru_14,ru,30.0,6.0,6.0,70.00,82.0,39.0,3.8,7.0
...,...,...,...,...,...,...,...,...,...,...,...
601,rum_67,rum_67,ru,40.0,69.0,38.0,93.75,88.0,52.0,4.0,11.0
602,rum_68,rum_68,ru,39.0,65.0,37.0,76.65,81.0,46.0,3.8,11.0
603,rum_69,rum_69,ru,37.0,66.0,37.0,68.75,84.0,51.0,3.6,9.0
604,rum_70,rum_70,ru,39.0,40.0,35.0,73.75,74.0,41.0,3.5,8.0


In [45]:

print(
    f" age-gender =  {len(age_gender_ru_subject_id)} \n", 
    f"ru_demo    =  {len(demo_ru_subject_id)} \n",
    f"fixation   = {len(ru_data_subject_id)} \n", 
)


 age-gender =  107 
 ru_demo    =  110 
 fixation   = 110 



In [62]:


print(
    
    f" missing in age_gender or demo    :  {age_gender_ru_subject_id.symmetric_difference(demo_ru_subject_id)} \n",
    f"missing in age_gender or fixation: {age_gender_ru_subject_id.symmetric_difference(ru_data_subject_id)} \n",
    f"missing in fixaiton or demo      : {ru_data_subject_id.symmetric_difference(demo_ru_subject_id)} \n",
    
)  



 missing in age_gender or demo    :  {'ru_9', 'rum_66', 'ru_4', 'ru_8', 'ru_2'} 
 missing in age_gender or fixation: {'ru_9', 'rum_66', 'ru_4', 'ru_8', 'ru_2'} 
 missing in fixaiton or demo      : set() 



In [63]:



print(
    
    f" missing in demo vs age_gender    :  {demo_ru_subject_id.difference(age_gender_ru_subject_id)} \n",
    f"missing in fixation vs age_gender:  {ru_data_subject_id.difference(age_gender_ru_subject_id)} \n",

)





 missing in demo vs age_gender    :  {'ru_9', 'ru_2', 'ru_4', 'ru_8'} 
 missing in fixation vs age_gender:  {'ru_9', 'ru_2', 'ru_4', 'ru_8'} 



In [65]:

print(
    
    f" missing in age_gender vs demo   :  {age_gender_ru_subject_id.difference(demo_ru_subject_id)} \n",
    f"missing in age_gender vs fixation:  {age_gender_ru_subject_id.difference(ru_data_subject_id)} \n",

)



 missing in age_gender vs demo   :  {'rum_66'} 
 missing in age_gender vs fixation:  {'rum_66'} 



In [68]:



print(
    
    f" missing in fixation vs demo   :  {ru_data_subject_id.difference(demo_ru_subject_id)} \n",
    f"missing in demo vs fixation   :  {demo_ru_subject_id.difference(ru_data_subject_id)} \n",


)



 missing in fixation vs demo   :  set() 
 missing in demo vs fixation   :  set() 



## midterm conclusion:

- We should drop out "{'ru_9', 'rum_66', 'ru_4', 'ru_8', 'ru_2'}" subjects from our experiments.

